# Week 9 - Macro Facility Basics (Part 1)

* Creating macro variables using the %LET statement
* Referencing macro variables and displaying their values
* System-defined vs. user-defined macro variables
* Local symbol table vs. global symnbol table

[Jupyter Python Notebook Keyboard Shortcuts and Text Snippets for Beginners](http://maxmelnick.com/2016/04/19/python-beginner-tips-and-tricks.html)


### Macro Facility = Macro Processor + Macro Language

### Why Using Macro Language?

#### One advantage is code substitution.

Instead of repeating the same piece(s) of text in your SAS program, 

* assign that text to a macro variable  [%LET DS = CLASS;] once
* and then just reference  it [&DS] at different parts of your SAS program without you having to type the same text over and over again.

The value of the macro variable could be:

* data set name
* data value
* variable name
* SAS statement 
* part of the SAS statement

Consider the macro code in the next cell and undestand about two token sequences (also called macro triggers).

* %LET ... (a macro statement)
* &dsn, &HowMany (&name-tokens also called macro variable reference)

* When the %LET statement is executed, the text values of the macro variable is stored in the global symbol table.

* The macro trigger is passed to the macro processor to search the symbol table for the reference. The macro processor resolves the macro variable references (&dsn and &HowMany) substituting their respective values (sashelp.class, and 5, respectively), passing the resolved value to the Input Stack.

* %PUT \_user_; is used to display the names and values all users-defined macro variables.

* SYMBOLGEN displays the results of resolving macro variable references in SAS log. This option is useful for debugging.

In the code snippet below, 

* you have both macro language and SAS language statements. For this code, the macro processor will execute macro language statements and then generate SAS code before any SAS language statements are executed.

In [1]:
options nocenter nodate nonumber symbolgen;
%let dsn = SASHELP.CLASS;
%let HowMany = 5;
%put _user_;
proc means data=&dsn; 
run;
title "Data Set: &dsn";
proc print data=&dsn (obs = &HowMany); 
run;

SAS Connection established. Subprocess id is 2748



Variable,N,Mean,Std Dev,Minimum,Maximum
Age Height Weight,19 19 19,13.3157895 62.3368421 100.0263158,1.4926722 5.1270752 22.7739335,11.0000000 51.3000000 50.5000000,16.0000000 72.0000000 150.0000000
Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,Barbara,F,13,65.3,98.0
4,Carol,F,14,62.8,102.5
5,Henry,M,14,63.5,102.5


### Understandung the use of PUT vs. %PUT statement

In [3]:
*Ex3_putlog_PercentPut.sas;
options nodate nosource notes nonumber nocenter 
        leftmargin=0.5in symbolgen;
ods html close;
%LET Path=C:\SASCourse\Week9;
LIBNAME perma "&Path";
data work.stocks; 
 set sashelp.stocks END=last;
  count+1;
  if last then putlog 
     @5 "Note: Number of observations=" count;
 run;
/*old way to display the macro-variable-value */  
 %put Note: Macro Variable Path = &Path; 

 /*new way to display the macro-variable-value */
 %put Note: Macro variable &=Path; 

#### When macro variables are resolved, the leading and trailing blanks from their values are removed.

In [6]:
*Ex2_percent_let.sas;
options nonotes nodate nonumber nosource;
ods html close;
%LET CITY1=   Washington DC;
%put &=CITY1;
title;

### Quotation marks are part of the macro variable value.

In [4]:
%LET CITY2=   "Washington DC";
%put &=CITY2;

### When macro variables are resolved, embedded extra blanks from their values are not removed.

In [16]:
%LET CITY3=   Washington   DC;
%put &=CITY3;

### Mathematical expressions in macro variable values do not get evaluated.

In [24]:
%LET m_sum=2+2;
%put &=m_sum;

### With the %EVAL function, mathematical expressions … do get evaluated.

In [25]:
%LET msum_eval=%eval(2+2);
%put &=msum_eval;

### Only with the %SYSEVALF function, expressions with fractional numbers … get evaluated.

In [11]:
%LET CALC_FRAC=%SYSEVALF(5.5/2);
%put &=CALC_FRAC;

#### %SYSFUNC is a function that executes SAS functions. PUTN assigns the value based on the value in the first argument and the format in the second argument.

In [26]:
%LET amount=%sysfunc(putn(5000000, dollar14.));
%put &=amount;

In [5]:
options nodate nonumber nonotes nosource;
ods html close;
data _NULL_;
amount = put(5000000, dollar14.);
put amount=
run;

In [ ]:
%showLog

In [61]:
*Ex8_Global_Local.sas;
*Code Adapted from Carpenter (2016);
option nosymbolgen;
%LET Year_outside = 2005;

%macro one;
 %global Year_inside_one;
 %LET Year_inside_one = 2006;
 %PUT &Year_inside_one;
%mend one;

%macro two;
 %LET Year_inside_two = 2007;
 %PUT &Year_inside_two;
%mend two;

%macro last;
  %one
  %two
  %put &Year_outside &Year_inside_one; 
%mend last;
%last


### Look-up technique using a DATA step variable in WHERE expression

In [4]:
*Ex5_Lookup_mvar.sas (Part 1);
Options nocenter nodate nonumber;
Title "Look-up using a data step variable";
proc print data=sashelp.demographics;
var name pop;
where pop>200000000;
run;
title;

Obs,NAME,pop
16,UNITED STATES,"298,212,895"
143,CHINA,"1,323,344,591"
145,INDIA,"1,103,370,802"
146,INDONESIA,"222,781,487"


### Look-up technique using a macro variable that contains a numeric value in WHERE expression

A macro variable value with no quotes around it would be treated as a numeric value.

In [6]:
*Ex5_Lookup_mvar.sas (Part 2);
Options nocenter nodate nonumber;
%let size=200000000;
proc print data=sashelp.demographics noobs;
var name pop;
where pop>&size;
run;

NAME,pop
UNITED STATES,"298,212,895"
CHINA,"1,323,344,591"
INDIA,"1,103,370,802"
INDONESIA,"222,781,487"


### Look-up technique using a macro variable that contains a character value in WHERE expression

Here, we have used the value of a macro variable that is treated as a character value and placed double quotes around the macro variable.  

In [33]:
*Ex5_Lookup_mvar.sas (Part 3);
options Options nocenter nodate nonumber symbolgen;
%let c_name=QATAR;
proc print data=sashelp.demographics noobs;
var name pop;
where name = "&c_name";
run;

NAME,pop
QATAR,"812,842"


### Look-up technique using a macro variable that contains a character value in WHERE expression

Here, we have used a macro variable, whose value is placed in single quotes when defining. No further quotes are needed in WHERE expression.

In [7]:
*Ex5_Lookup_mvar.sas (Part 4);
options nocenter nodate nonumber;
%let c_name='QATAR';
proc print data=sashelp.demographics noobs;
var name pop;
where name = &c_name;
run;


NAME,pop
QATAR,"812,842"


List obseravtions having the country names that begin  with Q or Z in the the SASHELP.DEMOGRAPHICS data set.

[Using IN:( ) to Code Character Comparisons with Criteria Having Different Lengths by Paul Grant (2009)](http://support.sas.com/resources/papers/proceedings09/056-2009.pdf)
  

In [12]:
*Ex5_Lookup_mvar.sas (Part 5);
options nosource nonotes symbolgen;
ods html close;
%let name_QZ=('Q', 'Z');
proc print data=sashelp.demographics noobs;
var name pop;
where name in: &name_QZ;
run;


NAME,pop
ZAIRE,"57,548,744"
ZIMBABWE,"13,009,534"
ZAMBIA,"11,668,457"
QATAR,"812,842"


In [13]:
%showLog

### Macro variable reference after text is not a problem.

In [39]:
*Ex6_Join_macro_var_text.sas (Part 1);
options nosymbolgen;
%LET dsn = month;
title;
data work.&dsn;
var1=1;
run;
%put work.&dsn;


###  Trailing text after the macro variable reference is a problem. In this situation, you must put a dot after the macro variable reference.

In [16]:
*Ex6_Join_macro_var_text.sas (Part 2);
options symbolgen;
%LET dsn = month;
 data work.&dsn.1;
var1=1;
run;
%put &dsn.1;

### Adjacent macro variable references are never a problem.

In [46]:
*Ex6_Join_macro_var_text.sas (Part 3);
*Join two macro variables;
%LET dsn = month;
%LET xdsn=1;
data work.&dsn&xdsn;
var1=1;
run;
%put work.&dsn&xdsn;


### Two dots are required because the single dot in the two-level name is considered as text after the first macro variable reference.

In [17]:
*Ex6_Join_macro_var_text.sas (Part 4);
options nonotes;
%LET Path=C:\Users\pmuhuri\Documents;
%LET Libref=New;
libname &Libref "&Path";
data &Libref..&dsn;
var1=1;
run;

In [51]:
*Ex6_Join_macro_var_text.sas (Part 5);
*Display GLOBAL macro variables;
%put _GLOBAL_;

*Delete user-defined macro variables;
%symdel Path Libref /nowarn;

* and check all macro variables deleted;
%put _GLOBAL_;